# Data Processing

## 1 - Load and Read Data

In [40]:
import findspark
findspark.init()

In [41]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('data_processing').getOrCreate()
df = spark.read.csv('sample_data.csv', inferSchema=True, header=True)

In [3]:
df.columns

['ratings', 'age', 'experience', 'family', 'mobile']

In [4]:
len(df.columns)

5

`printSchema` method of spark. It shows the datatypes of the columns along 
with the column names.

In [5]:
df.printSchema()

root
 |-- ratings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: double (nullable = true)
 |-- family: integer (nullable = true)
 |-- mobile: string (nullable = true)



The nullable property indicates if the corresponding column can 
assume null values (true) or not (false). We can also change the datatype of 
the columns as per the requirement

 We can use the Spark `show` method to view the top rows of the 
dataframe.

In [6]:
df.show(3)

+-------+---+----------+------+-------+
|ratings|age|experience|family| mobile|
+-------+---+----------+------+-------+
|      3| 32|       9.0|     3|   Vivo|
|      3| 27|      13.0|     3|  Apple|
|      4| 22|       2.5|     0|Samsung|
+-------+---+----------+------+-------+
only showing top 3 rows



In order to view only certain 
columns, we have to use the `select` method. Let us view only two columns 
(age and mobile)

In [7]:
df.select('age', 'mobile').show(5)

+---+-------+
|age| mobile|
+---+-------+
| 32|   Vivo|
| 27|  Apple|
| 22|Samsung|
| 37|  Apple|
| 27|     MI|
+---+-------+
only showing top 5 rows



The next function to be used is `describe` for analyzing the dataframe. It 
returns the statistical measures for each column of the dataframe

In [8]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------+
|summary|           ratings|               age|        experience|            family|mobile|
+-------+------------------+------------------+------------------+------------------+------+
|  count|                33|                33|                33|                33|    33|
|   mean|3.5757575757575757|30.484848484848484|10.303030303030303|1.8181818181818181|  null|
| stddev|1.1188806636071336|  6.18527087180309| 6.770731351213326|1.8448330794164254|  null|
|    min|                 1|                22|               2.5|                 0| Apple|
|    max|                 5|                42|              23.0|                 5|  Vivo|
+-------+------------------+------------------+------------------+------------------+------+



## 2 - Adding a New Column


We can add a new column in the dataframe using the `withColumn` function 
of spark

In [9]:
df = df.withColumn("age_after_10_yrs", (df["age"] + 10))

To change the datatype of the age column from integer to double, 
we can make use of the `cast` method in Spark. We need to import the 
`DoubleType` from `pyspark.types`

In [10]:
from pyspark.sql.types import StringType, DoubleType

df.withColumn("age_double", df['age'].cast(DoubleType())).show(10, False)

+-------+---+----------+------+-------+----------------+----------+
|ratings|age|experience|family|mobile |age_after_10_yrs|age_double|
+-------+---+----------+------+-------+----------------+----------+
|3      |32 |9.0       |3     |Vivo   |42              |32.0      |
|3      |27 |13.0      |3     |Apple  |37              |27.0      |
|4      |22 |2.5       |0     |Samsung|32              |22.0      |
|4      |37 |16.5      |4     |Apple  |47              |37.0      |
|5      |27 |9.0       |1     |MI     |37              |27.0      |
|4      |27 |9.0       |0     |Oppo   |37              |27.0      |
|5      |37 |23.0      |5     |Vivo   |47              |37.0      |
|5      |37 |23.0      |5     |Samsung|47              |37.0      |
|3      |22 |2.5       |0     |Apple  |32              |22.0      |
|3      |27 |6.0       |0     |MI     |37              |27.0      |
+-------+---+----------+------+-------+----------------+----------+
only showing top 10 rows



## 3- Filtering Data

Filtering records based on conditions is a common requirement when 
dealing with data. This helps in cleaning the data and keeping only 
relevant records. Filtering in PySpark is pretty straight-forward and can be 
done using the `filter` function

**Condition 1**

This is the most basic type of filtering based on only one column of a 
dataframe. Let us say we want to fetch the records for only ‘Vivo’ mobile:

In [11]:
df.filter(df['mobile'] == 'Vivo').show()

+-------+---+----------+------+------+----------------+
|ratings|age|experience|family|mobile|age_after_10_yrs|
+-------+---+----------+------+------+----------------+
|      3| 32|       9.0|     3|  Vivo|              42|
|      5| 37|      23.0|     5|  Vivo|              47|
|      4| 37|       6.0|     0|  Vivo|              47|
|      5| 37|      13.0|     1|  Vivo|              47|
|      4| 37|       6.0|     0|  Vivo|              47|
+-------+---+----------+------+------+----------------+



In [12]:
df.filter(df['mobile'] == 'Vivo').select('age', 'ratings', 'mobile').show()

+---+-------+------+
|age|ratings|mobile|
+---+-------+------+
| 32|      3|  Vivo|
| 37|      5|  Vivo|
| 37|      4|  Vivo|
| 37|      5|  Vivo|
| 37|      4|  Vivo|
+---+-------+------+



**Condition 2**

This involves multiple columns-based filtering and returns records only 
if all conditions are met. This can be done in multiple ways. Let us say, we 
want to filter only ‘Vivo’ users and only those with experience of more than 
10 years

In [13]:
df.filter(df['mobile']=='Vivo').filter(df['experience'] >10).show()

+-------+---+----------+------+------+----------------+
|ratings|age|experience|family|mobile|age_after_10_yrs|
+-------+---+----------+------+------+----------------+
|      5| 37|      23.0|     5|  Vivo|              47|
|      5| 37|      13.0|     1|  Vivo|              47|
+-------+---+----------+------+------+----------------+



There is another approach to achieve the same 
results as mentioned below.

In [14]:
df.filter((df['mobile']=='Vivo')&(df['experience'] >10)).show()

+-------+---+----------+------+------+----------------+
|ratings|age|experience|family|mobile|age_after_10_yrs|
+-------+---+----------+------+------+----------------+
|      5| 37|      23.0|     5|  Vivo|              47|
|      5| 37|      13.0|     1|  Vivo|              47|
+-------+---+----------+------+------+----------------+



## 4 - Distinct Values in Column

If we want to view the distinct values for any dataframe column, we can 
use the `distinct` method.

In [15]:
df.select('mobile').distinct().show()

+-------+
| mobile|
+-------+
|     MI|
|   Oppo|
|Samsung|
|   Vivo|
|  Apple|
+-------+



## 5 - Grouping Data

`Grouping` is a very useful way to understand various aspects of the 
dataset. It helps to group the data based on columns values and extract 
insights. It can be used with multiple other functions as well. Let us see an 
example of the `groupBy` method using the dataframe

In [16]:
df.groupBy('mobile').count().show(5,False)

+-------+-----+
|mobile |count|
+-------+-----+
|MI     |8    |
|Oppo   |7    |
|Samsung|6    |
|Vivo   |5    |
|Apple  |7    |
+-------+-----+



In [17]:
df.groupBy('mobile').count().orderBy('count',ascending=False).show(5,False)

+-------+-----+
|mobile |count|
+-------+-----+
|MI     |8    |
|Apple  |7    |
|Oppo   |7    |
|Samsung|6    |
|Vivo   |5    |
+-------+-----+



We can also apply the `groupBy` method to calculate statistical measures 
such as mean value, sum, min, or max value for each category.

In [18]:
df.groupBy('mobile').mean().show(5,False)

+-------+------------------+------------------+------------------+------------------+---------------------+
|mobile |avg(ratings)      |avg(age)          |avg(experience)   |avg(family)       |avg(age_after_10_yrs)|
+-------+------------------+------------------+------------------+------------------+---------------------+
|MI     |3.5               |30.125            |10.1875           |1.375             |40.125               |
|Oppo   |2.857142857142857 |28.428571428571427|10.357142857142858|1.4285714285714286|38.42857142857143    |
|Samsung|4.166666666666667 |28.666666666666668|8.666666666666666 |1.8333333333333333|38.666666666666664   |
|Vivo   |4.2               |36.0              |11.4              |1.8               |46.0                 |
|Apple  |3.4285714285714284|30.571428571428573|11.0              |2.7142857142857144|40.57142857142857    |
+-------+------------------+------------------+------------------+------------------+---------------------+



## 6 - Aggregations

We can use the `agg` function as well to achieve the similar kinds of results 
as above.

In [19]:
df.groupBy('mobile').agg({'experience':'sum'}).show(5,False)

+-------+---------------+
|mobile |sum(experience)|
+-------+---------------+
|MI     |81.5           |
|Oppo   |72.5           |
|Samsung|52.0           |
|Vivo   |57.0           |
|Apple  |77.0           |
+-------+---------------+



## 7 - User-Defined Functions (UDFs)

**UDFs** are widely used in data processing to apply certain transformations 
to the dataframe. There are two types of UDFs available in PySpark: 
**Conventional UDF** and **Pandas UDF**. Pandas UDF are much more powerful 
in terms of speed and processing time.

In [20]:
from pyspark.sql.functions import udf

**Traditional Python Function**

In [21]:
def price_range(brand):
    if brand in ['Samsung', 'Apple']:
        return 'High Price'
    elif brand == 'MI':
        return 'Mid Price'
    else:
        return 'Low Price'

We create a UDF (brand_udf) that uses this function 
and also captures its datatype to apply this tranformation on the mobile 
column of the dataframe

In [22]:
brand_udf=udf(price_range,StringType())

In [23]:
df.withColumn('price_range',brand_udf(df['mobile'])).show(10,False)

+-------+---+----------+------+-------+----------------+-----------+
|ratings|age|experience|family|mobile |age_after_10_yrs|price_range|
+-------+---+----------+------+-------+----------------+-----------+
|3      |32 |9.0       |3     |Vivo   |42              |Low Price  |
|3      |27 |13.0      |3     |Apple  |37              |High Price |
|4      |22 |2.5       |0     |Samsung|32              |High Price |
|4      |37 |16.5      |4     |Apple  |47              |High Price |
|5      |27 |9.0       |1     |MI     |37              |Mid Price  |
|4      |27 |9.0       |0     |Oppo   |37              |Low Price  |
|5      |37 |23.0      |5     |Vivo   |47              |Low Price  |
|5      |37 |23.0      |5     |Samsung|47              |High Price |
|3      |22 |2.5       |0     |Apple  |32              |High Price |
|3      |27 |6.0       |0     |MI     |37              |Mid Price  |
+-------+---+----------+------+-------+----------------+-----------+
only showing top 10 rows



**Using Lambda Function**

Instead of defining a traditional Python function, we can make use of the 
lambda function and create a UDF in a single line of code as shown below. 

In [24]:
age_udf = udf(lambda age: "young" if age <= 30 else "senior", StringType())
df.withColumn("age_group", age_udf(df.age)).show(10,False)

+-------+---+----------+------+-------+----------------+---------+
|ratings|age|experience|family|mobile |age_after_10_yrs|age_group|
+-------+---+----------+------+-------+----------------+---------+
|3      |32 |9.0       |3     |Vivo   |42              |senior   |
|3      |27 |13.0      |3     |Apple  |37              |young    |
|4      |22 |2.5       |0     |Samsung|32              |young    |
|4      |37 |16.5      |4     |Apple  |47              |senior   |
|5      |27 |9.0       |1     |MI     |37              |young    |
|4      |27 |9.0       |0     |Oppo   |37              |young    |
|5      |37 |23.0      |5     |Vivo   |47              |senior   |
|5      |37 |23.0      |5     |Samsung|47              |senior   |
|3      |22 |2.5       |0     |Apple  |32              |young    |
|3      |27 |6.0       |0     |MI     |37              |young    |
+-------+---+----------+------+-------+----------------+---------+
only showing top 10 rows



**Pandas UDF (Vectorized UDF)**

Like mentioned earlier, Pandas UDFs are way faster and efficient 
compared to their peers. There are two types of Pandas UDFs:
- Scalar
- GroupedMap

In [25]:
from pyspark.sql.functions import pandas_udf

In [26]:
from pyspark.sql.types import IntegerType

In this example, we define a Python function that calculates the 
number of years left in a user’s life assuming a life expectancy of 100 years. 

In [27]:
def remaining_yrs(age):
    yrs_left = (100-age)
    return yrs_left

In [28]:
length_udf = pandas_udf(remaining_yrs, IntegerType())

In [29]:
df.withColumn("yrs_left", length_udf(df['age'])).show(10, False)

+-------+---+----------+------+-------+----------------+--------+
|ratings|age|experience|family|mobile |age_after_10_yrs|yrs_left|
+-------+---+----------+------+-------+----------------+--------+
|3      |32 |9.0       |3     |Vivo   |42              |68      |
|3      |27 |13.0      |3     |Apple  |37              |73      |
|4      |22 |2.5       |0     |Samsung|32              |78      |
|4      |37 |16.5      |4     |Apple  |47              |63      |
|5      |27 |9.0       |1     |MI     |37              |73      |
|4      |27 |9.0       |0     |Oppo   |37              |73      |
|5      |37 |23.0      |5     |Vivo   |47              |63      |
|5      |37 |23.0      |5     |Samsung|47              |63      |
|3      |22 |2.5       |0     |Apple  |32              |78      |
|3      |27 |6.0       |0     |MI     |37              |73      |
+-------+---+----------+------+-------+----------------+--------+
only showing top 10 rows



**Pandas UDF (Multiple Columns)**

We might face a situation where we need multiple columns as input to 
create a new column. Hence, the below example showcases the method 
of applying a Pandas UDF on multiple columns of a dataframe

In [30]:
def prod(rating,exp):
    x = rating*exp
    return x

In [31]:
prod_udf = pandas_udf(prod, DoubleType())

In [32]:
df.withColumn("product", prod_udf(df['ratings'], df['experience'])).show(10, False)

+-------+---+----------+------+-------+----------------+-------+
|ratings|age|experience|family|mobile |age_after_10_yrs|product|
+-------+---+----------+------+-------+----------------+-------+
|3      |32 |9.0       |3     |Vivo   |42              |27.0   |
|3      |27 |13.0      |3     |Apple  |37              |39.0   |
|4      |22 |2.5       |0     |Samsung|32              |10.0   |
|4      |37 |16.5      |4     |Apple  |47              |66.0   |
|5      |27 |9.0       |1     |MI     |37              |45.0   |
|4      |27 |9.0       |0     |Oppo   |37              |36.0   |
|5      |37 |23.0      |5     |Vivo   |47              |115.0  |
|5      |37 |23.0      |5     |Samsung|47              |115.0  |
|3      |22 |2.5       |0     |Apple  |32              |7.5    |
|3      |27 |6.0       |0     |MI     |37              |18.0   |
+-------+---+----------+------+-------+----------------+-------+
only showing top 10 rows



## 8 - Drop Duplicate Values

We can use the `dropDuplicates` method in order to remove the duplicate 
records from the dataframe.

In [33]:
df.count()

33

In [34]:
df = df.dropDuplicates()
df.count()

26

## 9 - Delete Column

We can make use of the `drop` function to remove any of the columns 
from the dataframe.

In [35]:
df_new = df.drop('mobile')
df_new.show()

+-------+---+----------+------+----------------+
|ratings|age|experience|family|age_after_10_yrs|
+-------+---+----------+------+----------------+
|      4| 37|      16.5|     4|              47|
|      3| 42|      23.0|     5|              52|
|      4| 37|       6.0|     0|              47|
|      5| 37|      23.0|     5|              47|
|      2| 27|       9.0|     2|              37|
|      4| 22|       2.5|     0|              32|
|      5| 27|       6.0|     0|              37|
|      4| 27|       9.0|     0|              37|
|      3| 22|       2.5|     0|              32|
|      2| 27|       6.0|     2|              37|
|      5| 27|       6.0|     2|              37|
|      3| 27|      13.0|     3|              37|
|      4| 37|       9.0|     2|              47|
|      4| 22|       6.0|     1|              32|
|      5| 22|       2.5|     0|              32|
|      4| 27|       6.0|     1|              37|
|      3| 32|       9.0|     3|              42|
|      5| 27|       

## 10 - Writing Data

Once we have the processing steps completed, we can write the clean 
dataframe to the desired location (local/cloud) in the required format.

**CSV**

In [36]:
df.printSchema()

root
 |-- ratings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: double (nullable = true)
 |-- family: integer (nullable = true)
 |-- mobile: string (nullable = true)
 |-- age_after_10_yrs: integer (nullable = true)



In [37]:
pwd

'D:\\Valdera Project\\Datsci\\Data Engineering\\pyspark\\01 - Data Processing'

In [39]:
df.toPandas().to_csv('out.csv', index=False)  